

# Proyecto

### Equipo:

- Juan Carlos Araya
- Javiera Palominos

- \<Nombre de usuarios en Codalab\>

- \<Nombre del Equipo en Codalab\>

### Link de repositorio de GitHub: `\<http://....\>`




## 1. Introducción

El objetivo de este proyecto consiste en ...

Los datos que proveen es un dataset con X ejemplos que describen una observación de ... 
Son N atributos y la variables objetivos son de tipo ... y ...

La primera tarea se evalua en base a la métrica ... ya que esta permite medir ...

Nuestra propuesta para resolver el problema consistieron en modelo basado en...

Nuestro modelo cumplió/no cumplió las expectativas...

---
## 2. Análisis Exploratorio de Datos

In [120]:
# Libreria Core
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# Libreria EDA
from scipy import stats
from pandas.api.types import is_numeric_dtype
from pandas.core.dtypes.common import is_datetime_or_timedelta_dtype
from IPython.display import display, Markdown, Latex

# Libreria para implementar Bag of Words
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

# Libreria Preprocessing - Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.feature_selection import SelectPercentile, f_classif, f_regression
from sklearn.model_selection import train_test_split

# Libreria Clasififcadores
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



# Libreria Regresores

# Libreria métricas de evaluación
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score

# Libreria Halving Grid Search
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kuanki\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kuanki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [121]:
df_train = pd.read_pickle("train.pickle")
df_train = df_train.astype(
    {
        "english": bool,
        "required_age": object,
        "short_description": str,
    }
)
display(df_train.head())

dims = df_train.shape

print("El DataFrame tiene", dims[0], "filas y", dims[1] ,"columnas")
print("Columnas Dataset: ", list(df_train.columns), "\n")
print("Tipos de Variables: ", "\n", df_train.dtypes, "\n")
print("Cantidad de valores nulos por columna:")
print(df_train.isna().sum())
print("Presenta un total de", len(df_train[df_train.duplicated(subset=["name"])]) ,"filas con nombres duplicados.")
print("Descripción numérica del Dataframe:")
display(df_train.describe(include=[np.number])) #cambiar object a number


#print("Descripción de las columnas: ")

# df_train = pd.read_csv(...)

# px.histograms(...)

#cov = df_train.cov()

#sns.heatmap(cov)

#ax = sns.heatmap(abs(df_train.corr()), annot=True)
# px.scatter(proyecciones)
# etc...

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,tags,achievements,average_playtime,price,short_description,estimated_sells,rating
0,An Aspie Life,2018-03-29,True,Bradley Hennessey;Joe Watson,EnderLost Studios,windows,0,Single-player;Steam Achievements,Adventure;Casual;Free to Play;Indie;Simulation,Free to Play;Adventure;Indie,23,0,0.00,One day your roommate Leaves for no reason. Yo...,3914,Mixed
1,GhostControl Inc.,2014-06-06,True,bumblebee,Application Systems Heidelberg,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Casual;Indie;Simulation;Strategy,Turn-Based;Indie;Simulation,53,65,10.99,Manage a team of ghosthunters and free London ...,10728,Mixed
2,Deponia,2012-08-06,True,Daedalic Entertainment,Daedalic Entertainment,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie,Adventure;Point & Click;Comedy,19,217,6.99,"In Deponia, the world has degenerated into a v...",635792,Positive
3,Atlas Reactor,2016-10-04,True,Trion Worlds,Trion Worlds,windows,0,Multi-player;Online Multi-Player;Steam Achieve...,Free to Play;Strategy,Free to Play;Multiplayer;Strategy,121,1240,0.00,SEASON 6 NOW LIVE! The battle for Atlas contin...,253864,Positive
4,CHUCHEL,2018-03-07,True,Amanita Design,Amanita Design,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Indie,Adventure;Indie;Casual,7,245,7.99,CHUCHEL is a comedy adventure game from the cr...,49818,Mostly Positive


El DataFrame tiene 7881 filas y 16 columnas
Columnas Dataset:  ['name', 'release_date', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'tags', 'achievements', 'average_playtime', 'price', 'short_description', 'estimated_sells', 'rating'] 

Tipos de Variables:  
 name                   object
release_date           object
english                  bool
developer              object
publisher              object
platforms              object
required_age           object
categories             object
genres                 object
tags                   object
achievements            int64
average_playtime        int64
price                 float64
short_description      object
estimated_sells         int64
rating               category
dtype: object 

Cantidad de valores nulos por columna:
name                 0
release_date         0
english              0
developer            0
publisher            0
platforms            0
required_age         

,achievements,average_playtime,price,estimated_sells
count,7881.000000,7881.000000,7881.000000,7.881000e+03
mean,43.170156,439.296790,8.431342,2.105767e+05
std,265.399206,3303.162083,8.755668,1.513926e+06
min,0.000000,0.000000,0.000000,3.600000e+03
25%,0.000000,0.000000,1.990000,9.724000e+03
50%,15.000000,27.000000,6.990000,2.150800e+04
75%,35.000000,251.000000,11.390000,7.357300e+04
max,9821.000000,190625.000000,78.990000,7.944129e+07


In [ ]:
def profile_serie(serie_in, n_samples=1000, random_state=42):
    serie = serie_in.copy()

    profile = pd.Series(dtype='object')
    profile["Type"] = serie.dtype
    profile = pd.concat([profile, serie.describe(datetime_is_numeric=True)])

    # profile = pd.Series([])

    if is_numeric_dtype(serie):
        profile["Negative"] = (serie < 0).sum()
        profile["Negative (%)"] = (
            str(round((serie < 0).sum() / len(serie) * 100, 2)) + " %"
        )
        profile["Zeros"] = (serie == 0).sum()
        profile["Zeros (%)"] = (
            str(round((serie == 0).sum() / len(serie) * 100, 2)) + " %"
        )
        profile["Kurt"] = serie.kurt()
        profile["Skew"] = serie.skew()

    profile[" "] = " "  # espacio

    profile["Missing cells"] = serie.isnull().sum()
    profile["Missing cells (%)"] = (
        str(round(serie.isnull().sum() / len(serie) * 100, 2)) + " %"
    )
    profile["Duplicate rows"] = serie.duplicated(False).sum()
    profile["Duplicate rows (%)"] = (
        str(round(serie.duplicated(False).sum() / len(serie) * 100, 2)) + " %"
    )
    profile["Total size in memory"] = str(serie.memory_usage(index=True)) + " bytes"

    # profile = pd.concat([profile, description])

    profile = profile.rename(
        index={
            "count": "Number of observations",
            "mean": "Mean",
            "std": "Std",
            "min": "Min",
            "max": "Max",
            "unique": "Unique",
            "top": "Top",
            "freq": "Freq",
        }
    )
    no_outliers_fig = None

    if is_numeric_dtype(serie):

        sampled_serie = serie.sample(n_samples, random_state=random_state)
        fig = px.histogram(
            sampled_serie, marginal="box", title=f"{serie.name} - With Outliers"
        )

        no_outliers = sampled_serie.loc[(np.abs(stats.zscore(sampled_serie)) < 3)]
        # zscore = https://es.wikipedia.org/wiki/Unidad_tipificada
        
        no_outliers_fig = px.histogram(
            no_outliers, marginal="box", title=f"{serie.name} - Without Outliers"
        )

    elif is_datetime_or_timedelta_dtype(serie):
        sampled_serie = serie.sample(n_samples, random_state=random_state)
        fig = px.histogram(sampled_serie, marginal="box", title=f"{serie.name}")
    
    else:
        count = (
            serie.value_counts()[0:100]
            .reset_index()
            .rename(columns={"index": serie.name, serie.name: "Count"})
        )
        fig = px.bar(
            x=count[serie.name].astype(str),
            y=count["Count"],
            title=f"100 Most common categories of {serie.name}",
        )
    display(Markdown(f'## {serie.name} Profile'))
    display(profile)
    fig.show()

    if no_outliers_fig:
        no_outliers_fig.show()

In [ ]:
numbers=['price','achievements','average_playtime']
for number in numbers:
    profile_serie(X[number])

In [ ]:
#categories for



Al efectuar el análisis exploratorio de datos pudimos notar lo siguiente:

El atributo A tenía X datos nulos y Y duplicados... que podrían potencialmente dificultar el análisis.

La variable A se distribuye de forma normal. En cambio la B en forma exponencial. La variable C es categórica.

Los atributos A y B interactúan de P forma, mientras que los C y D de Q forma...

Los atributos A y C se correlacionan con las variables de salida Y de forma creciente... B no tiene correlación con la salida.

Además, encontramos que al proyectar en 2D ...

...

---

## 3. Preparación de Datos

Para preparar nuestros datos, primero eliminamos duplicados y outliers...:

In [122]:
## Código
x = df_train.iloc[:,:14]
y = df_train.iloc[:,14:16]

Después, imputamos ciertos datos

In [123]:
## Código
def SuperDummier(df_in, col):
    df = df_in.copy()
    df[col] = df[col].apply(lambda x: str(x).split(";"))
    s = pd.Series(df[col])
    l = pd.get_dummies(s.apply(pd.Series).stack(), prefix=col).max(level=0)
    df = pd.concat((df.drop(col,1),l),axis=1)
    return df

X = SuperDummier(x,"required_age")
X = SuperDummier(X,"platforms")
X = SuperDummier(X,"categories")
X = SuperDummier(X,"genres")
X = SuperDummier(X,"tags")

In [124]:
X

,name,release_date,english,developer,publisher,platforms,categories,genres,tags,achievements,average_playtime,price,short_description,required_age_0,required_age_12,required_age_16,required_age_18,required_age_3,required_age_7
0,An Aspie Life,2018-03-29,True,Bradley Hennessey;Joe Watson,EnderLost Studios,windows,Single-player;Steam Achievements,Adventure;Casual;Free to Play;Indie;Simulation,Free to Play;Adventure;Indie,23,0,0.00,One day your roommate Leaves for no reason. Yo...,1,0,0,0,0,0
1,GhostControl Inc.,2014-06-06,True,bumblebee,Application Systems Heidelberg,windows;mac;linux,Single-player;Steam Achievements;Steam Trading...,Casual;Indie;Simulation;Strategy,Turn-Based;Indie;Simulation,53,65,10.99,Manage a team of ghosthunters and free London ...,1,0,0,0,0,0
2,Deponia,2012-08-06,True,Daedalic Entertainment,Daedalic Entertainment,windows;mac;linux,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie,Adventure;Point & Click;Comedy,19,217,6.99,"In Deponia, the world has degenerated into a v...",1,0,0,0,0,0
3,Atlas Reactor,2016-10-04,True,Trion Worlds,Trion Worlds,windows,Multi-player;Online Multi-Player;Steam Achieve...,Free to Play;Strategy,Free to Play;Multiplayer;Strategy,121,1240,0.00,SEASON 6 NOW LIVE! The battle for Atlas contin...,1,0,0,0,0,0
4,CHUCHEL,2018-03-07,True,Amanita Design,Amanita Design,windows;mac,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Indie,Adventure;Indie;Casual,7,245,7.99,CHUCHEL is a comedy adventure game from the cr...,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7876,KOEWOTAYORINI / 声之寄托,2018-03-26,False,あみそ組,Pujia8 Studio;Gamera Game,windows,Single-player;Steam Achievements;Steam Cloud,Adventure;Free to Play;Indie;RPG,Free to Play;Adventure;Anime,20,65,0.00,“喂喂，求求你啦！救救我！” ——你接到这样一个电话 发出这样的求救，给你打电话的人，是一名...,1,0,0,0,0,0
7877,Montaro,2016-07-25,True,JCKSLAP,MBDL,windows,Single-player;Steam Achievements;Steam Trading...,Casual;Indie,Memes;Cute;Casual,15,174,0.79,Montaro is a DOGE.,1,0,0,0,0,0
7878,Moe Jigsaw,2018-03-23,True,ARES Inc.,ARES Inc.,windows,Single-player;Steam Achievements;Steam Trading...,Casual;Indie,Casual;Nudity;Indie,72,0,2.89,&quot;Moe Jigsaw&quot; is the definitive versi...,1,0,0,0,0,0
7879,Drunkn Bar Fight,2016-11-28,True,The Munky,The Munky,windows,Single-player;Multi-player;Online Multi-Player...,Action;Indie;Early Access,Early Access;Action;Indie,0,0,10.99,"VR PARTY GAMEDrunkn Bar Fight is a simple, imm...",1,0,0,0,0,0


Luego, preparamos un `ColumnTransformer` que prepara los datos:

In [125]:
stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

In [126]:
preprocessing = ColumnTransformer(
    transformers=[
        ('BagOfWords', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2)),'short_description'),
        ('StandardScaler', StandardScaler(),['required_age','price']), 
        ('MinMaxScaler',  MinMaxScaler(),['achievements','average_playtime'])
    ],
    remainder="drop"
)

---

## 4. Baseline

Adjuntamos las transformaciones anteriores en un solo `Pipeline` y agregamos al final un clasificador sencillo `ClasificadorX` 


In [127]:
baseline = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("RFC", DummyClassifier())
    ]
)

X_train, X_test, y_train, y_test = train_test_split(x, y.iloc[:,1:2], test_size=0.20)

baseline.fit(X_train, y_train)
y_pred = baseline.predict(X_test)
print(f1_score(y_test, y_pred,average='weighted'))
print(classification_report(y_test, y_pred))

0.2458238379072317
                 precision    recall  f1-score   support

          Mixed       0.36      0.13      0.19       342
Mostly Positive       0.21      0.10      0.14       297
       Negative       0.41      0.17      0.24       271
       Positive       0.30      0.84      0.44       425
  Very Positive       0.65      0.06      0.11       242

       accuracy                           0.31      1577
      macro avg       0.39      0.26      0.23      1577
   weighted avg       0.37      0.31      0.25      1577



In [128]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
#from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
#from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor


baseline_reg = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("RFC", XGBRegressor())
    ]
)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(x, y.iloc[:,0:1], test_size=0.20)

baseline_reg.fit(X_train_reg, y_train_reg)
y_pred_reg = baseline_reg.predict(X_test_reg)
print(r2_score(y_test_reg, y_pred_reg))

-0.14619292660518646


In [129]:
params = [
       {'clf': RandomForestClassifier(),
       'clf__n_estimators': [100,300, 1000],
       'selection': SelectPercentile(f_classif),
       'selection__percentile': [20, 40, 60, 80],
       'bog': CountVectorizer(tokenizer= StemmerTokenizer()),
       'Preprocessing__bog__ngram_range': [(1,1),(1,2), (1,3)],
       },
       ]

for param in params:
    clf = param['clf']
    param.pop('clf')

    selection = param['selection']
    param.pop('selection')

    bog = param['bog']
    param.pop('bog')

    preprocessing2 = ColumnTransformer(
    transformers=[
        ('bog', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2)),'short_description'),
        ('StandardScaler', StandardScaler(),['required_age','price']), 
        ('MinMaxScaler',  MinMaxScaler(),['achievements','average_playtime'])
    ],  
    remainder="drop"
    )

    pipe = Pipeline([("Preprocessing", preprocessing2),
    ("selection", selection),
    ("clf", clf)])

    grid = HalvingGridSearchCV(pipe, param_grid=param, random_state=0, min_resources=2, verbose=0)
    grid.fit(X_train, y_train)
    y_pred = grid.best_estimator_.predict(X_test)
    print(f"Clasificador {clf}")
    print(f"Mejores estimadores {grid.best_estimator_}")
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred,average="weighted"))

KeyboardInterrupt: 

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
print(f"Clasificador {clf}")
print(f"Mejores estimadores {grid.best_estimator_}")
print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred,average="weighted"))

Clasificador RandomForestClassifier()
Mejores estimadores Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('bog',
                                                  CountVectorizer(ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x0000027C4B582FD0>),
                                                  'short_description'),
                                                 ('StandardScaler',
                                                  StandardScaler(),
                                                  ['required_age', 'price']),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['achievements',
                                                   'average_playti

In [ ]:
## Código

# pipeline_clf = [('etapa1', etapa1), ..., ('clf', Clasificador)]
# pipeline_rg = [('etapa1', etapa1), ..., ('clf', Regresor)]

# Evalauación
# classfication_report(...)
# ....

---

## 5. Optimización del Modelo

Ahora, agregamos selección y/o reducción de atributos y probaremos una combinación de diferentes parámetros para mejorar nuestro modelo:


*Nota*: Procuren hacer Gridsearch sobre espacios de búsqueda razonables. Es decir, no estén 3 días buscando la mejor configuración...

In [ ]:
# grilla_parametros = [{'seleccion': SelectPercentile , 'percentile': [25, 50, 75], 'clf': clf_1}, 
#                      {'clf': clf_2, 'steps'; n},
#                      {...}, 
#                     ]
# gs_2 = HalvingGridSearchCV(pipeline_clf, grilla_parametros, metric='la_metrica_que_optimiza_el_problema')

Pudimos ver que mejor modelo mejora/no mejora los resultados obtenidos en el baseline.

In [ ]:
# gs_2.best_...

---

## 6. Conclusiones

Algunas respuestas que podrían plantearse pueden ser:

- ¿Pudieron resolver exitosamente el problema?
- ¿Son aceptables los resultados obtenidos?


- ¿Como fue el rendimiento del baseline?
- ¿Pudieron optimizar el baseline?
- ¿Que tanto mejoro el baseline con respecto a sus optimizaciones?


- ¿Estuvieron conformes con sus resultados?
- ¿Creen que hayan mejores formas de modelar el problema?
- ¿Creen que fue buena idea usar una competencia de por medio?
- ¿En general, qué aprendieron en el pryecto?¿Qué no aprendieron y les gustaría haber aprendido?
- Etc...

**OJO** si usted decide responder parte de estas preguntas, debe redactarlas en un formato de informe y no responderlas directamente.

---

<br>

### Anexo: Generación de Archivo Submit de la Competencia

Para subir los resultados obtenidos a la pagina de CodaLab utilice la función `generateFiles` entregada mas abajo. Esto es debido a que usted deberá generar archivos que respeten extrictamente el formato de CodaLab, de lo contario los resultados no se veran reflejados en la pagina de la competencia.

Para los resultados obtenidos en su modelo de clasificación y regresión, estos serán guardados en un archivo zip que contenga los archivos `predicctions_clf.txt` para la clasificación y `predicctions_rgr.clf` para la regresión. Los resultados, como se comento antes, deberan ser obtenidos en base al dataset `test.pickle` y en cada una de las lineas deberan presentar las predicciones realizadas.

Ejemplos de archivos:

- [ ] `predicctions_clf.txt`

        Mostly Positive
        Mostly Positive
        Negative
        Positive
        Negative
        Positive
        ...

- [ ] `predicctions_rgr.txt`

        16103.58
        16103.58
        16041.89
        9328.62
        107976.03
        194374.08
        ...



In [ ]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict(predict_data)
    y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")